<a href="https://colab.research.google.com/github/jose-marquez89/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

**Filepath for my sake and for later:** module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb

In [0]:
import pandas as pd
import numpy as np

In [3]:
# Get the data 
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-12-04 02:46:39--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [                 <=>]  21.69M   815KB/s    in 28s     

2019-12-04 02:47:08 (796 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22744357]



In [4]:
# Unzip the archive
!unzip LoanStats_2018Q4.csv.zip 

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [5]:
# Load the data set into a new pandas dataframe, skip last two rows and fix header
df = pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2, engine='python')
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,14777.99,14777.99,7481.000000,7481.00,...,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,25000,25000,25000.0,60 months,16.14%,609.82,C,C4,Production Manager,5 years,MORTGAGE,45000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,703xx,LA,37.09,0,Sep-2003,0,NaN,NaN,7,0,8901,36.8%,21,w,0.00,0.00,26653.167580,26653.17,...,33.3,0,0,73683,49524,18600,49483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,26500,26500,26500.0,60 months,11.31%,580.28,B,B3,Compliance Director,10+ years,MORTGAGE,134000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,747xx,OK,18.91,0,Dec-2006,2,NaN,NaN,17,0,43640,65.4%,37,w,23051.06,23051.06,5777.820000,5777.82,...,66.7,0,0,430403,107975,49300,88875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10400,10400,10400.0,36 months,12.98%,350.32,B,B5,Program Support Assistant,10+ years,MORTGAGE,56099.0,Source Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,800xx,CO,20.92,0,Jul-2013,2,32.0,67.0,8,1,1669,9.8%,10,w,7902.57,7902.57,3491.950000,3491.95,...,0.0,1,0,60709,39564,17000,43709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,35000,35000,35000.0,60 months,12.98%,796.00,B,B5,Sr. Vice President -Risk Manager,7 years,MORTGAGE,211646.0,Source Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,797xx,TX,15.09,0,Sep-1985,0,77.0,NaN,13,0,55944,74.9%,39,w,0.00,0.00,36266.741552,36266.74,...,66.7,0,0,516204,82514,59700,51904,88364.0,Sep-1985,0.0,6.0,19.0,53.4,2.0,29.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Change the "term" column from strings to integers

# Create a function to isolate numbers
def monthsToInt(cellContents):
  return int(cellContents.strip('months').strip())

In [0]:
# Apply function to entire "term" column
df['term'] = df['term'].apply(monthsToInt)

In [18]:
# "term" column is now comprised of numeric values
print(df['term'])
type(df['term'][1234])

0         36
1         60
2         60
3         36
4         60
          ..
128407    36
128408    36
128409    36
128410    36
128411    36
Name: term, Length: 128412, dtype: int64


numpy.int64

In [0]:
# Investigating loan_status

# Checking to see if the desired categories need cleaning
# There was probably a better way to do this?
[category for category in df['loan_status'][:30]]

['Current',
 'Fully Paid',
 'Current',
 'Current',
 'Fully Paid',
 'Fully Paid',
 'Current',
 'Fully Paid',
 'Current',
 'Current',
 'Fully Paid',
 'Fully Paid',
 'Current',
 'Current',
 'Current',
 'Current',
 'Current',
 'Current',
 'Fully Paid',
 'Current',
 'Current',
 'Current',
 'Current',
 'Current',
 'Current',
 'Current',
 'Fully Paid',
 'Current',
 'Charged Off',
 'Current']

In [0]:
# Create a function to handle the loan_status category conversions
def clear(cellContents):
  if cellContents == 'Current' or cellContents == 'Fully Paid':
    return 1
  else:
    return 0

In [0]:
# Create the new column using the new function
df['loan_status_is_great'] = df['loan_status'].apply(clear)

# Side by side comparison for verification
df[['loan_status','loan_status_is_great']][:50]

,loan_status,loan_status_is_great
0,Current,1
1,Fully Paid,1
2,Current,1
3,Current,1
4,Fully Paid,1
5,Fully Paid,1
6,Current,1
7,Fully Paid,1
8,Current,1
9,Current,1


In [0]:
# Create last_pymnt_d_month
byMonth = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)
df['last_pymnt_d_month'] = byMonth.dt.month

In [0]:
# New column is created but results in float values 
df['last_pymnt_d_month']

0         11.0
1          4.0
2         10.0
3         10.0
4          4.0
          ... 
128407    12.0
128408    11.0
128409    11.0
128410    11.0
128411    11.0
Name: last_pymnt_d_month, Length: 128412, dtype: float64

In [0]:
# A search on stack overflow revealed that this could be due to NaN values
df['last_pymnt_d'].value_counts(dropna=False)

# TODO: go back and fill NaN values with something else

Nov-2019    94238
Oct-2019    10607
Jul-2019     3156
Sep-2019     2871
Aug-2019     2758
May-2019     2547
Apr-2019     2356
Jun-2019     2285
Mar-2019     1984
Feb-2019     1915
Jan-2019     1561
Dec-2018     1054
Nov-2018      679
Oct-2018      234
NaN           159
Dec-2019        8
Name: last_pymnt_d, dtype: int64

In [0]:
# Repeating the process for year
byYear = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)
df['last_pymnt_d_year'] = byMonth.dt.year

In [0]:
# Now the two new columns are created

'''I proceeded in this way so that I didn't have to change the 
   original column to timestamps. Not really sure if that was
   necessary'''

df[['last_pymnt_d', 'last_pymnt_d_month', 'last_pymnt_d_year']][:20]

,last_pymnt_d,last_pymnt_d_month,last_pymnt_d_year
0,Nov-2019,11.0,2019.0
1,Apr-2019,4.0,2019.0
2,Oct-2019,10.0,2019.0
3,Oct-2019,10.0,2019.0
4,Apr-2019,4.0,2019.0
5,Feb-2019,2.0,2019.0
6,Oct-2019,10.0,2019.0
7,Sep-2019,9.0,2019.0
8,Oct-2019,10.0,2019.0
9,Nov-2019,11.0,2019.0


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01